In [1]:
import dash
from dash import dcc, html, dash_table
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output

In [2]:
df = px.data.tips()
df["tips_percentage"] = df['tip'] / df['total_bill'] * 100

In [3]:
df.head(5)

,total_bill,tip,sex,smoker,day,time,size,tips_percentage
0,16.99,1.01,Female,No,Sun,Dinner,2,5.944673
1,10.34,1.66,Male,No,Sun,Dinner,3,16.054159
2,21.01,3.50,Male,No,Sun,Dinner,3,16.658734
3,23.68,3.31,Male,No,Sun,Dinner,2,13.978041
4,24.59,3.61,Female,No,Sun,Dinner,4,14.680765


In [4]:
boxplot_dropdown_options = {
    'time': ['Dinner', 'Lunch'],
    'sex': ['Female', 'Male']
}

In [5]:
app = dash.Dash(__name__)

# App layout
app.layout = html.Div(children=[
    # Title
    html.H1(
        "Dashboard: restaurant",
        style={'textAlign': 'center', 'marginBottom': '30px'}
    ),
    # Top section with two charts
    html.Div(children=[
        # Bar chart on the left
        html.Div(children=[
            dcc.Slider(
                id='left_input',
                min=0,
                max=3,
                marks={0: 'Thur', 1: 'Fri', 2: 'Sat', 3: 'Sun'},
                step=None,
                value=0
            ),
            dcc.Graph(id='left_output')
        ], style={'display': 'inline-block', 'width': '48%', 'vertical-align': 'top'}),

        # Scatter plot on the right
        html.Div(children=[
            dcc.Dropdown(
                id='right_input',
                options=[
                    {'label': 'Female', 'value': 'Female'},
                    {'label': 'Male', 'value': 'Male'}
                ],
                value='Female'
            ),
            dcc.Graph(id='right_output')
        ], style={'display': 'inline-block', 'width': '48%', 'vertical-align': 'top'}),
    ], style={'display': 'flex', 'justify-content': 'space-between'}),

    # Bottom part with boxplot and table
    html.Div(children=[
        # Boxplot left
        html.Div(children=[
            html.Div(children=[
                dcc.Dropdown(
                    id='source_dropdown',
                    options=[
                        {'label': col, 'value': col} for col in ['sex', 'smoker', 'day', 'time']
                    ],
                    value='time'
                )
            ], style={'margin-bottom': '10px'}),
            html.Div(children=[
                dcc.Dropdown(id='subcategory_dropdown')
            ], style={'margin-bottom': '10px'}),
            dcc.Graph(id='boxplot')
        ], style={'display': 'inline-block', 'width': '48%', 'vertical-align': 'top'}),

        # Table on the right
        html.Div(children=[
            dash_table.DataTable(
                id='data_table',
                editable=True,
                export_format='csv',
                style_table={'height': '400px', 'overflowY': 'auto'},
                style_data={
                    'whiteSpace': 'normal',
                    'height': 'auto',
                    'textAlign': 'center'
                },
                style_data_conditional=[]
            )
        ], style={'display': 'inline-block', 'width': '48%', 'vertical-align': 'top'})
    ], style={'display': 'flex', 'justify-content': 'space-between', 'margin-top': '20px'})
])

# CALLBACK: Bar chart update
@app.callback(
    Output('left_output', 'figure'),
    Input('left_input', 'value')
)
def update_left_bar(selected_day):
    day_map = {0: 'Thur', 1: 'Fri', 2: 'Sat', 3: 'Sun'}
    day = day_map[selected_day]
    df_day = df[df['day'] == day]
    df_day_grouped = df_day.groupby('size', as_index=False)['total_bill'].sum()
    fig_bar = px.bar(
        df_day_grouped,
        x="size",
        y="total_bill",
        labels={"size": "Party Size", "total_bill": "Total Bill"},
        title=f"Total Bill by Party Size on {day}",
        height=350,
        template="plotly_white"
    )
    return fig_bar

# CALLBACK: Scatter plot update
@app.callback(
    Output('right_output', 'figure'),
    Input('right_input', 'value')
)
def update_right_scatter(selected_sex):
    df_sex = df[df['sex'] == selected_sex]
    fig = px.scatter(
        df_sex,
        x="tips_percentage",
        y="total_bill",
        labels={"tips_percentage": "Tips (%)", "total_bill": "Total Bill"},
        title=f"Tips Percentage vs Total Bill ({selected_sex})",
        range_x=[0, 80],
        range_y=[0, 55],
        height=350,
        template="plotly_white"
    )
    return fig

# CALLBACK: Boxplot update
@app.callback(
    Output('boxplot', 'figure'),
    [Input('subcategory_dropdown', 'value'),
     Input('source_dropdown', 'value')]
)
def update_boxplot(subcategory_dropdown, source_dropdown):
    df_sub_cat = df[df[source_dropdown] == subcategory_dropdown]
    fig_box = px.box(
        df_sub_cat,
        x="day",
        y="total_bill",
        labels={"day": "Day", "total_bill": "Total Bill"},
        category_orders={"day": ["Thur", "Fri", "Sat", "Sun"]},
        range_y=[0, 55],
        height=350,
        template="plotly_white"
    )
    return fig_box

# CALLBACK: Setting subcategory options in dropdown
@app.callback(
    Output('subcategory_dropdown', 'options'),
    Input('source_dropdown', 'value')
)
def set_sub_category_options(selected_main_category):
    return [{'label': i, 'value': i} for i in df[selected_main_category].unique()]

# CALLBACK: Set default subcategory value
@app.callback(
    Output('subcategory_dropdown', 'value'),
    Input('subcategory_dropdown', 'options')
)
def set_sub_category_value(available_options):
    return available_options[0]['value']

# CALLBACK: Update table with conditional formatting
@app.callback(
    [Output('data_table', 'data'),
     Output('data_table', 'columns'),
     Output('data_table', 'style_data_conditional')],
    [Input('source_dropdown', 'value'),
     Input('subcategory_dropdown', 'value')]
)
def update_table_with_formatting(selected_source, selected_subcategory):
    # Filter data based on source and subcategory selection
    filtered_df = df[df[selected_source] == selected_subcategory]

    # Pivot table with aggregation
    df_pivot = pd.pivot_table(
        filtered_df,
        index='day',
        columns='size',
        values='total_bill',
        aggfunc='sum'
    ).reindex(["Thur", "Fri", "Sat", "Sun"])

    # Reset index to display in table
    df_pivot.columns = df_pivot.columns.astype(str)  # Columns for string
    df_pivot.reset_index(inplace=True)               # Reset index for column 'day'

    # Setting table columns
    columns = [
        {"name": i, "id": i, "deletable": True, "hideable": True} for i in df_pivot.columns
    ]

    # Conditional table formatting (green and red based on values)
    style_data_conditional = []
    for column in df_pivot.columns[1:]:  # Omit the first column ('day')
        style_data_conditional.append({
            'if': {
                'filter_query': f'{{{column}}} >= 100',
                'column_id': column
            },
            'backgroundColor': 'green',
            'color': 'white'
        })
        style_data_conditional.append({
            'if': {
                'filter_query': f'{{{column}}} < 100',
                'column_id': column
            },
            'backgroundColor': 'lightcoral',
            'color': 'white'
        })

    return df_pivot.to_dict('records'), columns, style_data_conditional

if __name__ == '__main__':
    app.run_server(debug=True)